In [4]:
a = fread("human_gene_name.tsv")
a$n <- 1
fwrite(a, "human_gene_name.tsv", sep = "\t")

In [ ]:
sos run xqtl-protocol/code/data_preprocessing/phenotype/gene_annotation.ipynb annotate_coord \
    --cwd ./ \
    --phenoFile human_gene_name.tsv \
    --phenotype-id-column gene_name \
    --molecular-trait-type gene \
    --coordinate-annotation data/resource/references/Homo_sapiens.GRCh38.103.chr.reformatted.collapse_only.gene.ERCC.gtf

In [1]:
library(tidyverse)
library(data.table)

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.2     v tibble    3.2.1
v lubridate 1.9.4     v tidyr     1.3.1
v purrr     1.0.4     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: 'data.table'


The following objects are masked from 'package:lubridate':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from 'package:dplyr':

    between, first, last


The following object is masked from 'package:purrr':

    transpose




In [2]:
setwd("~/")
getwd()

[1] "/home/ubuntu"

## sQTL

In [34]:
meta_file = fread("project/image_QTL/DNE_cb_explained_by_xQTL/xQTL_top_loci.tsv")
meta_file = meta_file |> filter(Data_Type == "sQTL")
dim(meta_file)

[1] 3 5

In [35]:
# Assuming your meta_file dataframe is already loaded
# If not, you'd need to recreate it from your data

# Create a function to read a single file and add identifiers
read_topolci_file <- function(path, cohort, data_type, modality) {
  # Read the file - assuming tab-delimited with gzip compression
  data <- read_tsv(path, show_col_types = FALSE)
  
  # Add identifier columns
  data$Cohort <- cohort
  data$Data_Type <- data_type
  data$Modality <- modality
  
  # Create combined identifier
  data$Cohort_Data_Type_Modality <- paste(cohort, data_type, modality, sep = "_")
  
  return(data)
}

# Apply the function to each row in the meta_file
combined_data <- pmap_dfr(
  list(
    path = meta_file$Path,
    cohort = meta_file$Cohort,
    data_type = meta_file$Data_Type,
    modality = meta_file$Modality
  ),
  read_topolci_file
)

# Save the combined dataset if needed
#fwrite(combined_data, "combined_top_loci_data.tsv")

# Preview the result
head(combined_data)

#chr,start,end,a1,a2,variant_ID,gene_ID,event_ID,cs_coverage_0.95,cs_coverage_0.7,cs_coverage_0.5,PIP,conditional_effect,cluster,cat,ID,Cohort,Data_Type,Modality,Cohort_Data_Type_Modality
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,10396,10397,*,CCCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCCTCGCGGTACCCTCAG,chr1:10397:CCCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCCTCGCGGTACCCTCAG:*,ENSG00000158286,ROSMAP_AC_sQTL_chr1:6209539:6209928:clu_403_+:PR:ENSG00000158286,0,0,0,0.02901275,0.4162247,clu_403_,PR,chr1:6209539:6209928:clu_403_+:PR,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
1,10396,10397,*,CCCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCCTCGCGGTACCCTCAG,chr1:10397:CCCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCCTCGCGGTACCCTCAG:*,ENSG00000248333,ROSMAP_AC_sQTL_chr1:1637202:1637407:clu_6616_-:PR:ENSG00000248333,0,0,0,0.04186011,-0.6134922,clu_6616_,PR,chr1:1637202:1637407:clu_6616_-:PR,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
1,10432,10433,ACCCTTAACCCCTAACCCCTAACC,ACCCTAACC,chr1:10433:ACCCTAACC:ACCCTTAACCCCTAACCCCTAACC,ENSG00000116213,ROSMAP_AC_sQTL_chr1:3647560:3649930:clu_6810_-:PR:ENSG00000116213,0,0,0,0.08295409,-1.2588737,clu_6810_,PR,chr1:3647560:3649930:clu_6810_-:PR,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
1,10437,10438,*,AAC,chr1:10438:AAC:*,ENSG00000160087,ROSMAP_AC_sQTL_chr1:1256181:1256991:clu_6523_-:UP:ENSG00000160087,0,0,0,0.02530852,0.7696662,clu_6523_,UP,chr1:1256181:1256991:clu_6523_-:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
1,10439,10440,*,CCCCTAACCCTAACCCTAACCCTAACCCTCGCGGTACCCTCAG,chr1:10440:CCCCTAACCCTAACCCTAACCCTAACCCTCGCGGTACCCTCAG:*,ENSG00000078369,ROSMAP_AC_sQTL_chr1:1839238:1839628:clu_6656_-:NE:ENSG00000078369,0,0,0,0.02845004,-0.2279109,clu_6656_,NE,chr1:1839238:1839628:clu_6656_-:NE,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
1,10442,10443,TTAACCCTAACCCTAACCCT,CTAACCCTAACCCTAACCCT,chr1:10443:CTAACCCTAACCCTAACCCT:TTAACCCTAACCCTAACCCT,ENSG00000169972,ROSMAP_AC_sQTL_chr1:1309851:1310633:clu_127_+:PR:ENSG00000169972,0,0,0,0.02746254,-1.3924732,clu_127_,PR,chr1:1309851:1310633:clu_127_+:PR,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC


In [52]:
three_overlap = readLines("project/image_QTL/interested_genes/CB_M1/DDX17_CBY1_var.txt")
three_overlap <- gsub('"', '', three_overlap)

In [53]:
three_overlap

[1] "chr22:38578149:C:A"     "chr22:38590243:G:A"     "chr22:38592110:A:G"    
 [4] "chr22:38593808:A:G"     "chr22:38597462:A:G"     "chr22:38599380:G:T"    
 [7] "chr22:38602514:C:A"     "chr22:38609057:A:T"     "chr22:38617825:T:C"    
[10] "chr22:38625830:G:GC"    "chr22:38640275:T:C"     "chr22:38640764:T:C"    
[13] "chr22:38641699:T:TGTTG" "chr22:38649094:T:C"     "chr22:38652311:A:G"    
[16] "chr22:38657752:G:C"     "chr22:38676093:C:G"     "chr22:38677891:A:G"    
[19] "chr22:38684073:T:C"     "chr22:38686169:A:G"     "chr22:38703597:T:C"    
[22] "chr22:38705614:G:C"     "chr22:38705628:T:C"     "chr22:38708315:C:T"    
[25] "chr22:38709943:T:C"     "chr22:38717129:G:A"     "chr22:38717898:C:T"    
[28] "chr22:38720790:A:C"     "chr22:38733994:A:G"     "chr22:38734959:C:T"    
[31] "chr22:38737798:C:T"     "chr22:38738710:T:C"     "chr22:38741829:C:G"    
[34] "chr22:38743338:A:G"     "chr22:38745595:G:A"     "chr22:38750522:C:T"    
[37] "chr22:38751710:C:T"     "chr22:38752478:G:A"     "chr22:38754445:G:A"    
[40] "chr22:38754538:G:T"

In [59]:
SUN2_var = combined_data |> filter(variant_ID %in% three_overlap)
SUN2_var |> count(gene_ID)

gene_ID,n
<chr>,<int>
ENSG00000100211,5
ENSG00000100221,4
ENSG00000184949,109
ENSG00000198792,23


In [55]:
SUN2_var

#chr,start,end,a1,a2,variant_ID,gene_ID,event_ID,cs_coverage_0.95,cs_coverage_0.7,cs_coverage_0.5,PIP,conditional_effect,cluster,cat,ID,Cohort,Data_Type,Modality,Cohort_Data_Type_Modality
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
22,38592109,38592110,G,A,chr22:38592110:A:G,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38668132:38670766:clu_143018_+:UP:ENSG00000100211,0,0,0,0.021934321,-0.40321481,clu_143018_,UP,chr22:38668132:38670766:clu_143018_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38592109,38592110,G,A,chr22:38592110:A:G,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38670989:38673158:clu_143019_+:UP:ENSG00000100211,0,0,0,0.001049308,-0.25552230,clu_143019_,UP,chr22:38670989:38673158:clu_143019_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38592109,38592110,G,A,chr22:38592110:A:G,ENSG00000100221,ROSMAP_AC_sQTL_chr22:38700123:38700965:clu_144722_-:NE:ENSG00000100221,0,0,0,0.002629362,0.04866929,clu_144722_,NE,chr22:38700123:38700965:clu_144722_-:NE,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38597461,38597462,G,A,chr22:38597462:A:G,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38668132:38670766:clu_143018_+:UP:ENSG00000100211,0,0,0,0.001920936,-0.35379653,clu_143018_,UP,chr22:38668132:38670766:clu_143018_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38597461,38597462,G,A,chr22:38597462:A:G,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38670989:38673158:clu_143019_+:UP:ENSG00000100211,0,0,0,0.001279521,-0.27017207,clu_143019_,UP,chr22:38670989:38673158:clu_143019_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38617824,38617825,C,T,chr22:38617825:T:C,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38668132:38670766:clu_143018_+:UP:ENSG00000100211,0,0,0,0.001075891,-0.31508327,clu_143018_,UP,chr22:38668132:38670766:clu_143018_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38717128,38717129,A,G,chr22:38717129:G:A,ENSG00000100221,ROSMAP_AC_sQTL_chr22:38700123:38700965:clu_144722_-:NE:ENSG00000100221,0,0,0,0.002035720,0.04965771,clu_144722_,NE,chr22:38700123:38700965:clu_144722_-:NE,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38717897,38717898,T,C,chr22:38717898:C:T,ENSG00000100221,ROSMAP_AC_sQTL_chr22:38700123:38700965:clu_144722_-:NE:ENSG00000100221,0,0,0,0.002035720,0.04965771,clu_144722_,NE,chr22:38700123:38700965:clu_144722_-:NE,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38754444,38754445,A,G,chr22:38754445:G:A,ENSG00000100221,ROSMAP_AC_sQTL_chr22:38700123:38700965:clu_144722_-:NE:ENSG00000100221,0,0,0,0.002122077,0.04997759,clu_144722_,NE,chr22:38700123:38700965:clu_144722_-:NE,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC


In [56]:
CBY1_var = combined_data |> filter(variant_ID %in% three_overlap)

In [58]:
CBY1_var |> filter(gene_ID == "ENSG00000100211")

#chr,start,end,a1,a2,variant_ID,gene_ID,event_ID,cs_coverage_0.95,cs_coverage_0.7,cs_coverage_0.5,PIP,conditional_effect,cluster,cat,ID,Cohort,Data_Type,Modality,Cohort_Data_Type_Modality
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
22,38592109,38592110,G,A,chr22:38592110:A:G,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38668132:38670766:clu_143018_+:UP:ENSG00000100211,0,0,0,0.021934321,-0.4032148,clu_143018_,UP,chr22:38668132:38670766:clu_143018_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38592109,38592110,G,A,chr22:38592110:A:G,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38670989:38673158:clu_143019_+:UP:ENSG00000100211,0,0,0,0.001049308,-0.2555223,clu_143019_,UP,chr22:38670989:38673158:clu_143019_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38597461,38597462,G,A,chr22:38597462:A:G,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38668132:38670766:clu_143018_+:UP:ENSG00000100211,0,0,0,0.001920936,-0.3537965,clu_143018_,UP,chr22:38668132:38670766:clu_143018_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38597461,38597462,G,A,chr22:38597462:A:G,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38670989:38673158:clu_143019_+:UP:ENSG00000100211,0,0,0,0.001279521,-0.2701721,clu_143019_,UP,chr22:38670989:38673158:clu_143019_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC
22,38617824,38617825,C,T,chr22:38617825:T:C,ENSG00000100211,ROSMAP_AC_sQTL_chr22:38668132:38670766:clu_143018_+:UP:ENSG00000100211,0,0,0,0.001075891,-0.3150833,clu_143018_,UP,chr22:38668132:38670766:clu_143018_+:UP,ROSMAP,sQTL,AC,ROSMAP_sQTL_AC


In [67]:
DDX17_var = combined_data |> filter(variant_ID %in% three_overlap)

In [71]:
DDX17_var |> filter(gene_ID == "ENSG00000100201")

#chr,start,end,a1,a2,variant_ID,gene_ID,event_ID,cs_coverage_0.95,cs_coverage_0.7,cs_coverage_0.5,PIP,conditional_effect,cluster,cat,ID,Cohort,Data_Type,Modality,Cohort_Data_Type_Modality
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


## eQTL

In [28]:
meta_file = fread("project/image_QTL/DNE_cb_explained_by_xQTL/xQTL_top_loci.tsv")
meta_file = meta_file |> filter(Data_Type == "eQTL" & Cohort == "ROSMAP" & Method == "single_context_finemapping")
meta_file

Data_Type,Cohort,Modality,Method,Path
<chr>,<chr>,<chr>,<chr>,<chr>
eQTL,ROSMAP,AC,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/AC_DeJager_eQTL.exported.toploci.bed.gz
eQTL,ROSMAP,Ast_10_Kellis,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/Ast_10_Kellis_eQTL.exported.toploci.bed.gz
eQTL,ROSMAP,Ast_DeJager,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/Ast_DeJager_eQTL.exported.toploci.bed.gz
eQTL,ROSMAP,Ast_Kellis,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/Ast_Kellis_eQTL.exported.toploci.bed.gz
eQTL,ROSMAP,Ast_mega,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/Ast_mega_eQTL.exported.toploci.bed.gz
eQTL,ROSMAP,DLPFC,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/DLPFC_DeJager_eQTL.exported.toploci.bed.gz
eQTL,ROSMAP,Exc_DeJager,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/Exc_DeJager_eQTL.exported.toploci.bed.gz
eQTL,ROSMAP,Exc_Kellis,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/Exc_Kellis_eQTL.exported.toploci.bed.gz
eQTL,ROSMAP,Exc_mega,single_context_finemapping,data/analysis_result/single_context/ROSMAP_eQTL/export/summary/context_specific/Exc_mega_eQTL.exported.toploci.bed.gz


In [29]:
# Assuming your meta_file dataframe is already loaded
# If not, you'd need to recreate it from your data

# Create a function to read a single file and add identifiers
read_topolci_file <- function(path, cohort, data_type, modality) {
  # Read the file - assuming tab-delimited with gzip compression
  data <- read_tsv(path, show_col_types = FALSE)
  
  # Add identifier columns
  data$Cohort <- cohort
  data$Data_Type <- data_type
  data$Modality <- modality
  
  # Create combined identifier
  data$Cohort_Data_Type_Modality <- paste(cohort, data_type, modality, sep = "_")
  
  return(data)
}

# Apply the function to each row in the meta_file
combined_data <- pmap_dfr(
  list(
    path = meta_file$Path,
    cohort = meta_file$Cohort,
    data_type = meta_file$Data_Type,
    modality = meta_file$Modality
  ),
  read_topolci_file
)

# Save the combined dataset if needed
#fwrite(combined_data, "combined_top_loci_data.tsv")

# Preview the result
head(combined_data)

#chr,start,end,a1,a2,variant_ID,gene_ID,event_ID,cs_coverage_0.95,cs_coverage_0.7,cs_coverage_0.5,PIP,conditional_effect,Cohort,Data_Type,Modality,Cohort_Data_Type_Modality
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,99088,99089,*,TTTC,chr1:99089:TTTC:*,ENSG00000196581,AC_DeJager_eQTL_ENSG00000196581,0,0,0,0.14741504,0.3104743,ROSMAP,eQTL,AC,ROSMAP_eQTL_AC
1,600805,600806,C,G,chr1:600806:G:C,ENSG00000198912,AC_DeJager_eQTL_ENSG00000198912,0,0,0,0.23786542,-1.0948060,ROSMAP,eQTL,AC,ROSMAP_eQTL_AC
1,602262,602263,*,ATCC,chr1:602263:ATCC:*,ENSG00000187730,AC_DeJager_eQTL_ENSG00000187730,0,0,0,0.03324526,0.3465442,ROSMAP,eQTL,AC,ROSMAP_eQTL_AC
1,602277,602278,*,TCCCTCTCCTTTCTCCTCTCTAGCC,chr1:602278:TCCCTCTCCTTTCTCCTCTCTAGCC:*,ENSG00000187730,AC_DeJager_eQTL_ENSG00000187730,0,0,0,0.07769440,0.3721831,ROSMAP,eQTL,AC,ROSMAP_eQTL_AC
1,610727,610728,T,C,chr1:610728:C:T,ENSG00000127054,AC_DeJager_eQTL_ENSG00000127054,0,0,0,0.04314056,-0.4321317,ROSMAP,eQTL,AC,ROSMAP_eQTL_AC
1,618361,618362,A,G,chr1:618362:G:A,ENSG00000196581,AC_DeJager_eQTL_ENSG00000196581,0,0,0,0.02578836,-0.0710322,ROSMAP,eQTL,AC,ROSMAP_eQTL_AC


In [31]:
a = combined_data |> filter(str_detect(event_ID, "ENSG00000100211"))
a |> filter(variant_ID %in% var_PR_id)

#chr,start,end,a1,a2,variant_ID,gene_ID,event_ID,cs_coverage_0.95,cs_coverage_0.7,cs_coverage_0.5,PIP,conditional_effect,Cohort,Data_Type,Modality,Cohort_Data_Type_Modality
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>


## TWAS

In [9]:
Aging2_chr6_1 = fread("project/image_QTL/twas/output/image_Aging2_ROSMAP_mega/twas/image_Aging2_ROSMAP_mega.chr6_145632539_148538997.twas.tsv.gz")
Aging2_chr6_2 = fread("project/image_QTL/twas/output/image_Aging2_ROSMAP_mega/twas/image_Aging2_ROSMAP_mega.chr6_148538997_149932268.twas.tsv.gz")
Aging2_chr6_3 = fread("project/image_QTL/twas/output/image_Aging2_ROSMAP_mega/twas/image_Aging2_ROSMAP_mega.chr6_149932268_151686826.twas.tsv.gz")
Aging2_chr6 = rbind(Aging2_chr6_1,Aging2_chr6_2, Aging2_chr6_3)

In [10]:
nrow(Aging2_chr6)
nrow(unique(Aging2_chr6))

[1] 336

[1] 336

In [12]:
Aging2_chr6 |> filter(molecular_id == "ENSG00000131023")

chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
6,ENSG00000131023,149718104,148560000,151480000,Ast_mega_eQTL,image_Aging2,enet,TRUE,FALSE,0.02530581,8.486454e-06,-3.138218,1.699786e-03,eQTL,chr6_148538997_149932268
6,ENSG00000131023,149718104,148560000,151480000,Ast_mega_eQTL,image_Aging2,lasso,TRUE,FALSE,0.02477297,1.047053e-05,-3.229598,1.239642e-03,eQTL,chr6_148538997_149932268
6,ENSG00000131023,149718104,148560000,151480000,Ast_mega_eQTL,image_Aging2,mrash,TRUE,FALSE,0.02988487,1.397724e-06,-3.104676,1.904874e-03,eQTL,chr6_148538997_149932268
6,ENSG00000131023,149718104,148560000,151480000,Ast_mega_eQTL,image_Aging2,susie,TRUE,TRUE,0.03304048,4.036291e-07,-4.256185,2.079445e-05,eQTL,chr6_148538997_149932268
6,ENSG00000131023,149718104,148560000,151480000,Exc_mega_eQTL,image_Aging2,enet,TRUE,TRUE,0.42404575,2.395158e-90,-3.832782,1.267021e-04,eQTL,chr6_148538997_149932268
6,ENSG00000131023,149718104,148560000,151480000,Exc_mega_eQTL,image_Aging2,lasso,TRUE,FALSE,0.42232592,7.179285e-90,-3.686415,2.274355e-04,eQTL,chr6_148538997_149932268
6,ENSG00000131023,149718104,148560000,151480000,Exc_mega_eQTL,image_Aging2,mrash,TRUE,FALSE,0.41283187,2.902168e-87,-3.333666,8.570952e-04,eQTL,chr6_148538997_149932268
6,ENSG00000131023,149718104,148560000,151480000,Exc_mega_eQTL,image_Aging2,susie,TRUE,FALSE,0.41557044,5.189962e-88,-3.892614,9.916999e-05,eQTL,chr6_148538997_149932268
6,ENSG00000131023,149718104,148560000,151480000,Inh_mega_eQTL,image_Aging2,enet,TRUE,FALSE,0.13022984,2.820083e-24,-3.953333,7.707016e-05,eQTL,chr6_148538997_149932268


## Aging

In [39]:
library(data.table)
TL1_mega_twas_files <- list.files(path = "~/project/image_QTL/twas/output/TL1_ROSMAP_mega_eQTL/twas/", 
                   pattern = "*twas\\.tsv\\.gz$", 
                   full.names = TRUE)

TL1_mega <- rbindlist(lapply(TL1_mega_twas_files, fread), fill = TRUE)

TL1_sQTL_twas_files <- list.files(path = "~/project/image_QTL/twas/output/TL1_ROSMAP_sQTL/twas", 
                   pattern = "*twas\\.tsv\\.gz$", 
                   full.names = TRUE)

TL1_sQTL <- rbindlist(lapply(TL1_sQTL_twas_files, fread), fill = TRUE)


In [42]:
Aging_twas = rbind(TL1_mega, TL1_sQTL)

In [43]:
head(Aging_twas)
dim(Aging_twas)

chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
11,ENSG00000006118,60924459,57280000,61937159,Ast_mega_eQTL,Telomere_pc1,enet,TRUE,TRUE,0.023858848,1.501658e-05,0.2312132,0.81714919,eQTL,chr11_60339997_63818332
11,ENSG00000006118,60924459,57280000,61937159,Ast_mega_eQTL,Telomere_pc1,lasso,TRUE,FALSE,0.023716018,1.588723e-05,0.1721534,0.86331694,eQTL,chr11_60339997_63818332
11,ENSG00000006118,60924459,57280000,61937159,Ast_mega_eQTL,Telomere_pc1,mrash,TRUE,FALSE,0.007431520,1.092498e-02,0.2230752,0.82347697,eQTL,chr11_60339997_63818332
11,ENSG00000006118,60924459,57280000,61937159,Ast_mega_eQTL,Telomere_pc1,susie,TRUE,FALSE,0.005449514,2.516846e-02,-0.4099552,0.68183884,eQTL,chr11_60339997_63818332
11,ENSG00000006118,60924459,57280000,61937159,Exc_mega_eQTL,Telomere_pc1,enet,TRUE,TRUE,0.023515616,1.719477e-05,-2.2395139,0.02512250,eQTL,chr11_60339997_63818332
11,ENSG00000006118,60924459,57280000,61937159,Exc_mega_eQTL,Telomere_pc1,lasso,TRUE,FALSE,0.023512908,1.721316e-05,-2.2019772,0.02766692,eQTL,chr11_60339997_63818332


[1] 4260   16

In [45]:
# DDX17
Aging_twas |> filter(molecular_id == "ENSG00000100201" & twas_pval < 2.5*1e-6)


chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
22,ENSG00000100201,38507659,37483438,41680000,Ast_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,0.019062446,1.000798e-04,5.428025,5.698118e-08,eQTL,chr22_37320379_39179317
22,ENSG00000100201,38507659,37483438,41680000,Ast_mega_eQTL,Telomere_pc1,lasso,TRUE,TRUE,0.021047402,4.559495e-05,5.311042,1.090002e-07,eQTL,chr22_37320379_39179317
22,ENSG00000100201,38507659,37483438,41680000,Ast_mega_eQTL,Telomere_pc1,susie,TRUE,FALSE,0.012163139,1.576276e-03,5.334323,9.590157e-08,eQTL,chr22_37320379_39179317
22,ENSG00000100201,38507659,37483438,41680000,OPC_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,0.009223728,5.264166e-03,5.187813,2.127776e-07,eQTL,chr22_37320379_39179317
22,ENSG00000100201,38507659,37483438,41680000,OPC_mega_eQTL,Telomere_pc1,lasso,TRUE,FALSE,0.006441291,1.665710e-02,5.045670,4.519353e-07,eQTL,chr22_37320379_39179317
22,ENSG00000100201,38507659,37483438,41680000,OPC_mega_eQTL,Telomere_pc1,susie,TRUE,TRUE,0.015436120,4.307650e-04,4.989920,6.040438e-07,eQTL,chr22_37320379_39179317
22,ENSG00000100201,38507659,37483438,41680000,Oli_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,0.033225769,3.752391e-07,5.599208,2.153328e-08,eQTL,chr22_37320379_39179317
22,ENSG00000100201,38507659,37483438,41680000,Oli_mega_eQTL,Telomere_pc1,lasso,TRUE,FALSE,0.033063037,4.000612e-07,5.363123,8.179526e-08,eQTL,chr22_37320379_39179317
22,ENSG00000100201,38507659,37483438,41680000,Oli_mega_eQTL,Telomere_pc1,mrash,TRUE,FALSE,0.031033760,8.892309e-07,4.850726,1.230104e-06,eQTL,chr22_37320379_39179317


In [46]:
# CBY1
Aging_twas |> filter(molecular_id == "ENSG00000100211"& twas_pval < 2.5*1e-6)

chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
22,ENSG00000100211,38656639,37520000,41680000,Exc_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,0.238812323,1.018713e-45,-5.867020,4.436963e-09,eQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,Exc_mega_eQTL,Telomere_pc1,lasso,TRUE,FALSE,0.236834588,2.654450e-45,-5.856108,4.738389e-09,eQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,Exc_mega_eQTL,Telomere_pc1,mrash,TRUE,FALSE,0.253007271,9.800927e-49,-5.531585,3.173506e-08,eQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,Exc_mega_eQTL,Telomere_pc1,susie,TRUE,TRUE,0.260047041,2.979502e-50,-5.422526,5.876251e-08,eQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,Inh_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,0.056296531,4.239173e-11,-5.867491,4.424385e-09,eQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,Inh_mega_eQTL,Telomere_pc1,lasso,TRUE,FALSE,0.054366786,9.118908e-11,-5.995605,2.027294e-09,eQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,Inh_mega_eQTL,Telomere_pc1,susie,TRUE,FALSE,0.059171760,1.351464e-11,-5.521437,3.362387e-08,eQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,Oli_mega_eQTL,Telomere_pc1,susie,TRUE,FALSE,0.015505797,4.120187e-04,-4.797165,1.609271e-06,eQTL,chr22_37320379_39179317
22,ENSG00000100211,38656639,37520000,41680000,ROSMAP_AC_sQTL_chr22:38670989:38673158:clu_143019_+:UP:ENSG00000100211,Telomere_pc1,enet,TRUE,FALSE,0.045599817,7.144613e-08,-5.831229,5.502052e-09,unproductive_sQTL,chr22_37320379_39179317


In [47]:
# FAM227A
Aging_twas |> filter(molecular_id == "ENSG00000184949" & twas_pval < 2.5*1e-6) 


chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
22,ENSG00000184949,38656628,37520000,41680000,Ast_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,0.071446837,9.607935e-14,5.641079,1.689877e-08,eQTL,chr22_37320379_39179317
22,ENSG00000184949,38656628,37520000,41680000,Ast_mega_eQTL,Telomere_pc1,lasso,TRUE,FALSE,0.072335605,6.718119e-14,5.581480,2.384802e-08,eQTL,chr22_37320379_39179317
22,ENSG00000184949,38656628,37520000,41680000,Ast_mega_eQTL,Telomere_pc1,mrash,TRUE,FALSE,0.060193789,8.716063e-12,5.413322,6.186624e-08,eQTL,chr22_37320379_39179317
22,ENSG00000184949,38656628,37520000,41680000,Ast_mega_eQTL,Telomere_pc1,susie,TRUE,TRUE,0.073476967,4.241631e-14,5.478104,4.299066e-08,eQTL,chr22_37320379_39179317
22,ENSG00000184949,38656628,37520000,41680000,Exc_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,0.304052408,4.519029e-60,5.858964,4.657631e-09,eQTL,chr22_37320379_39179317
22,ENSG00000184949,38656628,37520000,41680000,Exc_mega_eQTL,Telomere_pc1,lasso,TRUE,FALSE,0.301898946,1.411444e-59,5.928996,3.047920e-09,eQTL,chr22_37320379_39179317
22,ENSG00000184949,38656628,37520000,41680000,Inh_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,0.189531652,1.304690e-35,5.859709,4.636784e-09,eQTL,chr22_37320379_39179317
22,ENSG00000184949,38656628,37520000,41680000,Inh_mega_eQTL,Telomere_pc1,lasso,TRUE,FALSE,0.190959897,6.804336e-36,5.724860,1.035191e-08,eQTL,chr22_37320379_39179317
22,ENSG00000184949,38656628,37520000,41680000,Inh_mega_eQTL,Telomere_pc1,mrash,TRUE,TRUE,0.194830723,1.159071e-36,5.448021,5.093325e-08,eQTL,chr22_37320379_39179317


In [48]:
#tomm22
Aging_twas |> filter(molecular_id == "ENSG00000100216")
## slight effect

chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_AC_sQTL_chr22:38682095:38682326:clu_143020_+:UP:ENSG00000100216,Telomere_pc1,enet,TRUE,TRUE,0.019697963,3.207700e-04,-1.66428959,0.09605457,unproductive_sQTL,chr22_37320379_39179317
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_AC_sQTL_chr22:38682095:38682326:clu_143020_+:UP:ENSG00000100216,Telomere_pc1,lasso,TRUE,FALSE,0.018227320,5.193575e-04,-1.62178401,0.10484960,unproductive_sQTL,chr22_37320379_39179317
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_AC_sQTL_chr22:38682095:38682326:clu_143020_+:UP:ENSG00000100216,Telomere_pc1,mrash,TRUE,FALSE,-0.001650429,9.284111e-01,-2.12058203,0.03395699,unproductive_sQTL,chr22_37320379_39179317
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_AC_sQTL_chr22:38682095:38682326:clu_143020_+:UP:ENSG00000100216,Telomere_pc1,susie,TRUE,FALSE,-0.001629170,8.852787e-01,0.24963483,0.80286976,unproductive_sQTL,chr22_37320379_39179317
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_PCC_sQTL_chr22:38682441:38682878:clu_131092_+:PR:ENSG00000100216,Telomere_pc1,enet,TRUE,FALSE,0.033722842,5.364268e-05,-0.06833674,0.94551758,productive_sQTL,chr22_37320379_39179317
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_PCC_sQTL_chr22:38682441:38682878:clu_131092_+:PR:ENSG00000100216,Telomere_pc1,lasso,TRUE,TRUE,0.037880091,1.947481e-05,0.37752475,0.70578368,productive_sQTL,chr22_37320379_39179317
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_PCC_sQTL_chr22:38682441:38682878:clu_131092_+:PR:ENSG00000100216,Telomere_pc1,mrash,TRUE,FALSE,0.013544095,7.766477e-03,0.88117197,0.37822475,productive_sQTL,chr22_37320379_39179317
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_PCC_sQTL_chr22:38682441:38682878:clu_131092_+:PR:ENSG00000100216,Telomere_pc1,susie,TRUE,FALSE,0.022430653,8.507467e-04,2.34699277,0.01892562,productive_sQTL,chr22_37320379_39179317
22,ENSG00000100216,38681956,37520000,41680000,ROSMAP_PCC_sQTL_chr22:38682996:38683766:clu_131093_+:PR:ENSG00000100216,Telomere_pc1,enet,TRUE,FALSE,NA,NA,-1.75669674,0.07896951,productive_sQTL,chr22_37320379_39179317


In [50]:
#JOSD1
Aging_twas |> filter(molecular_id == "ENSG00000100221")


chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
22,ENSG00000100221,38701555,37520000,41680000,Ast_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,0.0010705453,1.814903e-01,NA,NA,eQTL,chr22_37320379_39179317
22,ENSG00000100221,38701555,37520000,41680000,Ast_mega_eQTL,Telomere_pc1,lasso,TRUE,FALSE,-0.0009040159,5.627641e-01,1.5752305,1.152033e-01,eQTL,chr22_37320379_39179317
22,ENSG00000100221,38701555,37520000,41680000,Ast_mega_eQTL,Telomere_pc1,mrash,TRUE,FALSE,0.0100407895,3.729815e-03,2.1969467,2.802426e-02,eQTL,chr22_37320379_39179317
22,ENSG00000100221,38701555,37520000,41680000,Ast_mega_eQTL,Telomere_pc1,susie,TRUE,TRUE,0.0112560796,2.275184e-03,-0.5702446,5.685118e-01,eQTL,chr22_37320379_39179317
22,ENSG00000100221,38701555,37520000,41680000,Exc_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,0.0288968265,2.062307e-06,-5.1002411,3.392211e-07,eQTL,chr22_37320379_39179317
22,ENSG00000100221,38701555,37520000,41680000,Exc_mega_eQTL,Telomere_pc1,lasso,TRUE,TRUE,0.0330843345,3.967215e-07,-4.6296726,3.662443e-06,eQTL,chr22_37320379_39179317
22,ENSG00000100221,38701555,37520000,41680000,Exc_mega_eQTL,Telomere_pc1,mrash,TRUE,FALSE,0.0196148084,8.039795e-05,-4.7463142,2.071568e-06,eQTL,chr22_37320379_39179317
22,ENSG00000100221,38701555,37520000,41680000,Exc_mega_eQTL,Telomere_pc1,susie,TRUE,FALSE,0.0277602948,3.226394e-06,-5.5996111,2.148333e-08,eQTL,chr22_37320379_39179317


In [51]:
# SUN2
Aging_twas |> filter(molecular_id == "ENSG00000100242")



chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
22,ENSG00000100242,38794142,37520000,41680000,Exc_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,0.009066422,5.557979e-03,5.0287107,4.937887e-07,eQTL,chr22_37320379_39179317
22,ENSG00000100242,38794142,37520000,41680000,Exc_mega_eQTL,Telomere_pc1,lasso,TRUE,TRUE,0.011692147,1.906827e-03,5.0397530,4.661330e-07,eQTL,chr22_37320379_39179317
22,ENSG00000100242,38794142,37520000,41680000,Exc_mega_eQTL,Telomere_pc1,mrash,TRUE,FALSE,-0.001171303,7.094549e-01,4.0471100,5.185388e-05,eQTL,chr22_37320379_39179317
22,ENSG00000100242,38794142,37520000,41680000,Exc_mega_eQTL,Telomere_pc1,susie,TRUE,FALSE,-0.001360127,9.860392e-01,2.0053942,4.492092e-02,eQTL,chr22_37320379_39179317
22,ENSG00000100242,38794142,37520000,41680000,Oli_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,0.154820268,6.394145e-29,-2.1784913,2.936948e-02,eQTL,chr22_37320379_39179317
22,ENSG00000100242,38794142,37520000,41680000,Oli_mega_eQTL,Telomere_pc1,lasso,TRUE,FALSE,0.160302114,5.752480e-30,-1.8948615,5.811076e-02,eQTL,chr22_37320379_39179317
22,ENSG00000100242,38794142,37520000,41680000,Oli_mega_eQTL,Telomere_pc1,mrash,TRUE,FALSE,0.168811518,1.328129e-31,-0.7468285,4.551671e-01,eQTL,chr22_37320379_39179317
22,ENSG00000100242,38794142,37520000,41680000,Oli_mega_eQTL,Telomere_pc1,susie,TRUE,TRUE,0.180928661,5.816888e-34,-1.5756097,1.151158e-01,eQTL,chr22_37320379_39179317


In [52]:
# sec31a
Aging_twas |> filter(molecular_id == "ENSG00000138674")


chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
4,ENSG00000138674,82901165,81818509,83901166,Exc_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,0.0937494586,1.112063e-17,2.10634101,0.03517474,eQTL,chr4_81722770_83364947
4,ENSG00000138674,82901165,81818509,83901166,Exc_mega_eQTL,Telomere_pc1,lasso,TRUE,TRUE,0.0947837724,7.270803e-18,2.46929173,0.01353808,eQTL,chr4_81722770_83364947
4,ENSG00000138674,82901165,81818509,83901166,Exc_mega_eQTL,Telomere_pc1,mrash,TRUE,FALSE,0.0747629207,2.525130e-14,2.24234541,0.02493906,eQTL,chr4_81722770_83364947
4,ENSG00000138674,82901165,81818509,83901166,Exc_mega_eQTL,Telomere_pc1,susie,TRUE,FALSE,0.0592737985,1.257823e-11,0.10013818,0.92023462,eQTL,chr4_81722770_83364947
4,ENSG00000138674,82901165,81818509,83901166,Inh_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,0.0449209300,3.824171e-09,0.09964563,0.92062567,eQTL,chr4_81722770_83364947
4,ENSG00000138674,82901165,81818509,83901166,Inh_mega_eQTL,Telomere_pc1,lasso,TRUE,FALSE,0.0422102506,1.113443e-08,0.09295195,0.92594173,eQTL,chr4_81722770_83364947
4,ENSG00000138674,82901165,81818509,83901166,Inh_mega_eQTL,Telomere_pc1,mrash,TRUE,FALSE,0.0575291504,2.597549e-11,0.45195643,0.65130037,eQTL,chr4_81722770_83364947
4,ENSG00000138674,82901165,81818509,83901166,Inh_mega_eQTL,Telomere_pc1,susie,TRUE,TRUE,0.0590380304,1.425341e-11,-0.16985785,0.86512193,eQTL,chr4_81722770_83364947
4,ENSG00000138674,82901165,81818509,83901166,ROSMAP_AC_sQTL_chr4:82857764:82862533:clu_37822_-:PR:ENSG00000138674,Telomere_pc1,enet,TRUE,FALSE,0.0123291844,3.654364e-03,1.56704389,0.11710443,productive_sQTL,chr4_81722770_83364947


In [53]:
## ENOPH1
Aging_twas |> filter(molecular_id == "ENSG00000145293")


chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
4,ENSG00000145293,82430589,81360000,83480000,ROSMAP_PCC_sQTL_chr4:82430884:82447919:clu_32170_+:PR:ENSG00000145293,Telomere_pc1,enet,TRUE,FALSE,0.0098759269,0.0198016649,1.270013214,0.204079923,productive_sQTL,chr4_81722770_83364947
4,ENSG00000145293,82430589,81360000,83480000,ROSMAP_PCC_sQTL_chr4:82430884:82447919:clu_32170_+:PR:ENSG00000145293,Telomere_pc1,lasso,TRUE,TRUE,0.0148943363,0.0055278141,0.705916798,0.480239881,productive_sQTL,chr4_81722770_83364947
4,ENSG00000145293,82430589,81360000,83480000,ROSMAP_PCC_sQTL_chr4:82430884:82447919:clu_32170_+:PR:ENSG00000145293,Telomere_pc1,mrash,TRUE,FALSE,0.0134753472,0.0079025343,1.782478634,0.074671212,productive_sQTL,chr4_81722770_83364947
4,ENSG00000145293,82430589,81360000,83480000,ROSMAP_PCC_sQTL_chr4:82430884:82447919:clu_32170_+:PR:ENSG00000145293,Telomere_pc1,susie,TRUE,FALSE,0.0045346457,0.0818845794,1.926756048,0.054010039,productive_sQTL,chr4_81722770_83364947
4,ENSG00000145293,82430589,81360000,83480000,ROSMAP_PCC_sQTL_chr4:82430913:82447919:clu_32170_+:PR:ENSG00000145293,Telomere_pc1,enet,TRUE,FALSE,0.0136272457,0.0076050997,-0.002638133,0.997895077,productive_sQTL,chr4_81722770_83364947
4,ENSG00000145293,82430589,81360000,83480000,ROSMAP_PCC_sQTL_chr4:82430913:82447919:clu_32170_+:PR:ENSG00000145293,Telomere_pc1,lasso,TRUE,FALSE,0.0115175519,0.0129919265,0.146775094,0.883309544,productive_sQTL,chr4_81722770_83364947
4,ENSG00000145293,82430589,81360000,83480000,ROSMAP_PCC_sQTL_chr4:82430913:82447919:clu_32170_+:PR:ENSG00000145293,Telomere_pc1,mrash,TRUE,TRUE,0.0295437106,0.0001487575,-0.920552614,0.357284052,productive_sQTL,chr4_81722770_83364947
4,ENSG00000145293,82430589,81360000,83480000,ROSMAP_PCC_sQTL_chr4:82430913:82447919:clu_32170_+:PR:ENSG00000145293,Telomere_pc1,susie,TRUE,FALSE,0.0193185778,0.0018347317,-0.522293127,0.601466254,productive_sQTL,chr4_81722770_83364947
4,ENSG00000145293,82430589,81360000,83480000,ROSMAP_PCC_sQTL_chr4:82448021:82454721:clu_32171_+:PR:ENSG00000145293,Telomere_pc1,enet,TRUE,FALSE,0.0038574391,0.0988843176,-0.441744212,0.658674313,productive_sQTL,chr4_81722770_83364947


In [54]:
# LIN54
Aging_twas |> filter(molecular_id == "ENSG00000189308")


chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
4,ENSG00000189308,83012925,81909973,84012926,Exc_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,4.871312e-02,8.338056e-10,3.384705306,7.125471e-04,eQTL,chr4_81722770_83364947
4,ENSG00000189308,83012925,81909973,84012926,Exc_mega_eQTL,Telomere_pc1,lasso,TRUE,FALSE,4.962090e-02,5.820681e-10,3.409546613,6.507096e-04,eQTL,chr4_81722770_83364947
4,ENSG00000189308,83012925,81909973,84012926,Exc_mega_eQTL,Telomere_pc1,mrash,TRUE,FALSE,6.377944e-02,2.081647e-12,3.357038217,7.878222e-04,eQTL,chr4_81722770_83364947
4,ENSG00000189308,83012925,81909973,84012926,Exc_mega_eQTL,Telomere_pc1,susie,TRUE,TRUE,6.382689e-02,2.042518e-12,4.135735320,3.538198e-05,eQTL,chr4_81722770_83364947
4,ENSG00000189308,83012925,81909973,84012926,Inh_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,4.592220e-05,3.096135e-01,1.005595450,3.146102e-01,eQTL,chr4_81722770_83364947
4,ENSG00000189308,83012925,81909973,84012926,Inh_mega_eQTL,Telomere_pc1,lasso,TRUE,FALSE,-4.880736e-05,3.264725e-01,1.552812626,1.204679e-01,eQTL,chr4_81722770_83364947
4,ENSG00000189308,83012925,81909973,84012926,Inh_mega_eQTL,Telomere_pc1,mrash,TRUE,FALSE,1.699494e-02,2.296349e-04,2.079950686,3.753006e-02,eQTL,chr4_81722770_83364947
4,ENSG00000189308,83012925,81909973,84012926,Inh_mega_eQTL,Telomere_pc1,susie,TRUE,TRUE,2.044071e-02,5.860271e-05,-0.013339392,9.893570e-01,eQTL,chr4_81722770_83364947
4,ENSG00000189308,83012925,81909973,84012926,ROSMAP_PCC_sQTL_chr4:82910811:82916943:clu_32176_+:PR:ENSG00000189308,Telomere_pc1,enet,TRUE,FALSE,-2.198722e-03,8.959171e-01,-0.402235617,6.875106e-01,productive_sQTL,chr4_81722770_83364947


In [55]:
# H2AC11
Aging_twas |> filter(molecular_id == "ENSG00000196787")


chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>


In [56]:
Aging_twas |> filter(molecular_id == "ENSG00000275126")


chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>


In [57]:
# MYRF
Aging_twas |> filter(molecular_id == "ENSG00000124920")


chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
11,ENSG00000124920,61752635,60752635,64320000,Oli_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,0.04814449,1.044236e-09,1.506114,0.13203789,eQTL,chr11_60339997_63818332
11,ENSG00000124920,61752635,60752635,64320000,Oli_mega_eQTL,Telomere_pc1,lasso,TRUE,FALSE,0.04862405,8.637276e-10,1.535464,0.12467000,eQTL,chr11_60339997_63818332
11,ENSG00000124920,61752635,60752635,64320000,Oli_mega_eQTL,Telomere_pc1,mrash,TRUE,FALSE,0.05157611,2.682228e-10,2.391674,0.01677171,eQTL,chr11_60339997_63818332
11,ENSG00000124920,61752635,60752635,64320000,Oli_mega_eQTL,Telomere_pc1,susie,TRUE,TRUE,0.05310708,1.461310e-10,2.583300,0.00978603,eQTL,chr11_60339997_63818332


In [59]:
# TMEM258
Aging_twas |> filter(molecular_id == "ENSG00000134825" & twas_pval < 0.05)


chr,molecular_id,TSS,start,end,context,gwas_study,method,is_imputable,is_selected_method,rsq_cv,pval_cv,twas_z,twas_pval,type,block
<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
11,ENSG00000134825,61792801,60768501,64320000,Oli_mega_eQTL,Telomere_pc1,enet,TRUE,FALSE,0.08664250,2.038326e-16,2.867843,0.004132810,eQTL,chr11_60339997_63818332
11,ENSG00000134825,61792801,60768501,64320000,Oli_mega_eQTL,Telomere_pc1,lasso,TRUE,FALSE,0.08467556,4.543395e-16,2.779804,0.005439174,eQTL,chr11_60339997_63818332
11,ENSG00000134825,61792801,60768501,64320000,Oli_mega_eQTL,Telomere_pc1,mrash,TRUE,TRUE,0.10003251,8.360156e-19,2.421565,0.015453823,eQTL,chr11_60339997_63818332
11,ENSG00000134825,61792801,60768501,64320000,Oli_mega_eQTL,Telomere_pc1,susie,TRUE,FALSE,0.09733376,2.545723e-18,2.531672,0.011352008,eQTL,chr11_60339997_63818332
11,ENSG00000134825,61792801,60768501,64320000,ROSMAP_PCC_sQTL_chr11:61781824:61783497:clu_75208_+:PR:ENSG00000134825,Telomere_pc1,susie,TRUE,FALSE,0.01226309,1.074446e-02,-2.282619,0.022452850,productive_sQTL,chr11_60339997_63818332
11,ENSG00000134825,61792801,60768501,64320000,ROSMAP_PCC_sQTL_chr11:61781824:61783512:clu_75208_+:PR:ENSG00000134825,Telomere_pc1,enet,TRUE,FALSE,0.03164921,8.895984e-05,2.867105,0.004142452,productive_sQTL,chr11_60339997_63818332
11,ENSG00000134825,61792801,60768501,64320000,ROSMAP_PCC_sQTL_chr11:61781824:61783512:clu_75208_+:PR:ENSG00000134825,Telomere_pc1,susie,TRUE,FALSE,0.01562036,4.607696e-03,2.297322,0.021600415,productive_sQTL,chr11_60339997_63818332


In [ ]:
# 